### 이번 실습에서는 다음 내용들을 배웁니다.


- selenium을 이용하여 실제 opinet 데이터를 자동으로 가져오는 웹크롤러를 만들어봅니다.

## 2.2. 전국 지역별 유가 정보 얻기

- selenium을 사용할 때 해야되는 기본적인 함수가 2가지 있습니다.


- webdriver.Chrome()과 driver.get() 입니다.


- webdriver.Chrome()은 실제 chromedriver가 존재하는 위치에서 chromedriver를 실행할 수 있게 불러오는 함수입니다.


- driver.get()은 selenium이 제어하고 있는 chromedriver가 해당 URL로 접속하게 하는 함수입니다.


- 여기서 또 문제점이 발생합니다. 지역별 검색 페이지에 접속해서 해당 URL로 driver.get()을 시도해도 메인페이지로 접속이 되는 문제입니다. 이런 경우는 해당 웹페이지 관리자가 다른 subpage들도 direct로 접속하지 못하게 메인페이지로 자동 redirection을 만들어 놓은 경우입니다.


- 이를 해결하기 위해 메인페이지에 먼저 접속한 뒤에, 지역별 페이지로 들어갑니다.


- Opinet 코드를 살펴보니 이 기능이 javascript 함수를 호출하는 것으로 되어 있습니다.


- selenium에서는 execute_script() 함수가 이를 지원하기 때문에 execute_script("goSubPage(0,0,99)")를 이용하여 해당 페이지에 접속합니다.

![redirection](./webcrawling/opinet_redirection.png)

In [34]:
# 해당 경로에 있는 chromedriver를 불러옵니다.
from selenium import webdriver
import time
# opinet.co.kr에 접속합니다.

oilcheck = webdriver.Chrome()
#oilcheck.get('https://www.opinet.co.kr/searRgSelect.do')
oilcheck.get('https://www.opinet.co.kr/user/main/mainView.do')
# 2초 쉽니다. (페이지 로딩을 위해)
time.sleep(2)
# 메인 페이지에서 유가 가격 검색 페이지로 이동합니다. javascript를 활용합니다.
oilcheck.execute_script('goSubPage(0,0,99)')

- selenium에서 특정 웹페이지 원소를 접근하는 방법이 여러가지가 있습니다.


**1) find_element_by_tag_name**


**2) find_element_by_selector**


**3) find_element_by_xpath**


**4) find_element_by_id**


각자 취향에 맞게 선택할 수 있습니다. 때에 따라 CSS selector만 선택해야 할때가 있고, xpath만 선택해야할 때가 있는데 이는 복잡한 문제이므로 넘어가겠습니다.

보통은 xpath로 일단 시도해본 뒤, 안되면 CSS selector나 tag name으로 시도해보면 좋습니다.


- 우선 우리는 지역목록을 가져와야 합니다.


- 이는 페이지에 drop-down 메뉴를 선택하여 해당 목록을 가져온 뒤, 하나하나 눌러서 접속해야 합니다.


- 지역에서 처음에 "도"를 고르고 그 뒤에 "구"를 고른 뒤에 "조회" 버튼을 누르면 해당 정보를 가져올 수 있습니다


- 정보를 제대로 가져왔는지는 가운데 있는 지도가 제대로 로드가 되었는지 확인해보면 됩니다. (초반 몇 개만)

![get_price](./webcrawling/opinet_get_price.png)

In [35]:
from selenium.webdriver.common.by import By
dir(By)

['CLASS_NAME',
 'CSS_SELECTOR',
 'ID',
 'LINK_TEXT',
 'NAME',
 'PARTIAL_LINK_TEXT',
 'TAG_NAME',
 'XPATH',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

In [36]:
# 앞서 배웠던 방식으로 xpath를 얻어옵니다.
from selenium.webdriver.common.by import By

sido = oilcheck.find_element(By.XPATH, '//*[@id="SIDO_NM0"]')
sido_names = sido.find_elements(By.TAG_NAME, 'option')

sido_list = []
for sido_name in sido_names:
    sido_list.append(sido_name.get_attribute('value'))
# 앞서 배웠던 방식으로 tag을 찾아서 가져옵니다.
# tag에서 value들을 하나씩 뽑아냅니다.
sido_list = sido_list[1:]
sido_list

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원특별자치도',
 '충청북도',
 '충청남도',
 '전북특별자치도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

- 이제 시도에 대한 정보를 얻었으니, 시/군/구도 동일한 방식으로 얻어오도록 합니다!

In [37]:
# 시/군/구 목록 수집
sido = oilcheck.find_element(By.XPATH, '//*[@id="SIDO_NM0"]')
sido.send_keys('서울특별시')

In [38]:
# "조회" 버튼의 xpath를 찾아서 클릭합니다.
sigu = oilcheck.find_element(By.XPATH, '//*[@id="SIGUNGU_NM0"]')
sigu.send_keys('서초구')

In [48]:
# "엑셀저장" 버튼의 xpath를 찾아서 클릭합니다.
oilcheck.find_element(By.XPATH, '//*[@id="searRgSelect"]').click()
time.sleep(2)
oilcheck.find_element(By.XPATH, '//*[@id="templ_list0"]/div[7]/div/a').click()

### 모두 종합해서 하나의 코드로 만들어봅시다!

In [55]:

# 1. webdriver를 킨다.
# 해당 경로에 있는 chromedriver를 불러옵니다.
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

spending_time = 2
oilcheck = webdriver.Chrome()
#oilcheck.get('https://www.opinet.co.kr/searRgSelect.do')
oilcheck.get('https://www.opinet.co.kr/user/main/mainView.do')
# 2초 쉽니다. (페이지 로딩을 위해)
time.sleep(spending_time)

# 2. 지역별 주유소 찾기 접속
oilcheck.execute_script('goSubPage(0,0,99)')
time.sleep(spending_time)

sido = oilcheck.find_element(By.XPATH, '//*[@id="SIDO_NM0"]')
sido_names = sido.find_elements(By.TAG_NAME, 'option')

sido_list = []
for sido_name in sido_names:
    sido_list.append(sido_name.get_attribute('value'))
    
sido_list = sido_list[1:]

# 4. 원하는 지역으로 이동한다.
for sido_name in sido_list:
    sido = oilcheck.find_element(By.XPATH, '//*[@id="SIDO_NM0"]')
    sido.send_keys(sido_name)
    time.sleep(spending_time)
    sigu = oilcheck.find_element(By.XPATH, '//*[@id="SIGUNGU_NM0"]')
    sigu_names = sigu.find_elements(By.TAG_NAME, 'option')
    
    sigu_list = []
    for sigu_name in sigu_names:
        sigu_list.append(sigu_name.get_attribute('value'))
    
    sigu_list = sigu_list[1:]    
    for sigu_name in sigu_list:
        sigu = oilcheck.find_element(By.XPATH, '//*[@id="SIGUNGU_NM0"]')
        sigu.send_keys(sigu_name)
        time.sleep(spending_time)
        # 6. 얻어온 목록으로 반복문을 수행하면서, 조회를 누르고 엑셀저장을 누른다.
        oilcheck.find_element(By.XPATH, '//*[@id="searRgSelect"]').click()
        time.sleep(spending_time)
        oilcheck.find_element(By.XPATH, '//*[@id="templ_list0"]/div[7]/div/a').click()
        time.sleep(spending_time)